In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns

In [2]:
from glob import glob

In [3]:
import sqlite3

conn = sqlite3.connect('datasets/icwsm-full-5.db')
df = pd.read_sql("""
select * from hitsbase
""", conn)
df.shape

(87, 137)

In [4]:
nb_participants = []
p_countries = []
for i in range(2, 6):
    nb_participants.append(
        df[[
            f'nb_participants_{i}_1',
            f'nb_participants_{i}_2',
            f'nb_participants_{i}_3',
            f'nb_participants_{i}_4',
            f'nb_participants_{i}_5',
        ]].apply(list, axis=1).apply(lambda x: list(filter(None, x)))
    )
    p_countries.append(
        df[[
            f'participant_country_{i}_1',
            f'participant_country_{i}_2',
            f'participant_country_{i}_3',
            f'participant_country_{i}_4',
            f'participant_country_{i}_5',
        ]].apply(list, axis=1).apply(lambda x: list(filter(None, x)))
    )

nb_participants = pd.concat(nb_participants, ignore_index=True)
p_countries = pd.concat(p_countries, ignore_index=True)

In [5]:
datasets = df[[
    'dataset_name_2',
    'dataset_name_3',
    'dataset_name_5',
    'dataset_name_4',
]].melt().value

papers = df[[
    'paper_url_2',
    'paper_url_3',
    'paper_url_5',
    'paper_url_4',
]].melt().value

In [6]:
data = pd.DataFrame({
    'paper_url': papers,
    'datasets': datasets,
    'countries': p_countries,
    'nb_participants': nb_participants,
})

In [7]:
data['datasets'].str.lower().str.contains('telegram').sum()

1

In [8]:
platforms = glob('datasets/demographics/*.csv')
platforms = pd.Series(platforms).str.split('/').str[-1].str.replace('.csv', '').tolist()

In [9]:
results = pd.DataFrame()
discarded = []
for _, paper in data.drop_duplicates('paper_url').iterrows():
    if len(paper['countries']) == 1:
        # Only one country
        if paper['countries'] != ['N/A']:
            # One known country
            results = pd.concat([
                results,
                pd.DataFrame([{
                    'paper_id': paper['paper_url'],
                    'country_code': paper['countries'][0],
                    'nb_records': 1,
                }])
            ], ignore_index=True)
        else:
            # Unknown countries
            if paper['datasets'] is None:
                continue

            paper_dataset = paper['datasets'].lower()
            paper_id = paper['paper_url']
            if paper_dataset in platforms:
                # Only one dataset
                platform = paper_dataset
                demo = pd.read_csv(f"datasets/demographics/{platform}.csv")
                demo['nb_records'] = demo['nb_users'] / demo['nb_users'].sum()

                demo.insert(0, 'paper_id', paper_id)
                results = pd.concat([
                    results,
                    demo[['paper_id', 'country_code', 'nb_records']]
                ], ignore_index=True)
            else:
                # Multiple datasets
                dataset_list = paper_dataset.split(', ')
                temp = []
                for d in dataset_list:
                    if d in platforms:
                        demo = pd.read_csv(f"datasets/demographics/{d}.csv")
                        demo['nb_records'] = demo['nb_users'] / demo['nb_users'].sum()
                        demo = demo.assign(platform=d)
                        temp.append(demo)

                if temp != []:
                    # Averaging values across datasets
                    temp = pd.concat(temp).pivot_table(
                        index='platform',
                        columns='country_code',
                        values='nb_records',
                        fill_value=0
                    ).mean(axis=0).reset_index()
                    temp.columns = ['country_code', 'nb_records']
                    temp.insert(0, 'paper_id', paper_id)
                    results = pd.concat([
                        results,
                        temp[['paper_id', 'country_code', 'nb_records']]
                    ], ignore_index=True)
                else:
                    if _ == 63:
                        print("X")
                    discarded.append(paper)
    elif len(set(paper['countries'])) == 1:
        # Multiple datasets, one known country
        if set(paper['countries']) != {'N/A'}:
            results = pd.concat([
                results,
                pd.DataFrame([{
                    'paper_id': paper['paper_url'],
                    'country_code': paper['countries'][0],
                    'nb_records': 1,
                }])
            ], ignore_index=True)
        else:
            # All unknown countries
            # Multiple datasets
            paper_dataset = paper['datasets'].lower()
            paper_id = paper['paper_url']
            dataset_list = paper_dataset.strip().split(', ')
            temp = []
            for d in dataset_list:
                if d in platforms:
                    demo = pd.read_csv(f"datasets/demographics/{d}.csv")
                    demo['nb_records'] = demo['nb_users'] / demo['nb_users'].sum()
                    demo = demo.assign(platform=d)
                    temp.append(demo)

            if temp != []:
                # Averaging values across datasets
                temp = pd.concat(temp).pivot_table(
                    index='platform',
                    columns='country_code',
                    values='nb_records',
                    fill_value=0
                ).mean(axis=0).reset_index()
                temp.columns = ['country_code', 'nb_records']
                temp.insert(0, 'paper_id', paper_id)
                results = pd.concat([
                    results,
                    temp[['paper_id', 'country_code', 'nb_records']]
                ], ignore_index=True)
    elif 'N/A' not in paper['countries']:
        # All known countries
        rep = pd.Series(paper['nb_participants']).dropna().values
        rep /= rep.sum()
        results = pd.concat([
            results,
            pd.DataFrame({
                'paper_id': paper['paper_url'],
                'country_code': paper['countries'],
                'nb_records': rep,
            })
        ], ignore_index=True)
    else:
        # TODO Double check this assumption
        rep = pd.Series(paper['nb_participants']).dropna().values
        rep /= rep.sum()
        results = pd.concat([
            results,
            pd.DataFrame({
                'paper_id': paper['paper_url'],
                'country_code': paper['countries'],
                'nb_records': rep,
            })
        ], ignore_index=True)

In [12]:
results.to_csv('datasets/icwsm-full-normalized-v2.csv', index=False)

In [10]:
results['paper_id'].nunique()

302

In [11]:
results.groupby('country_code').agg({'nb_records': 'sum'}).nlargest(10, 'nb_records').style.format('{:.2f}')

,nb_records
country_code,
USA,137.25
IND,15.41
JPN,14.92
GBR,13.08
CHN,10.88
N/A,9.33
BRA,9.33
DEU,6.90
IDN,6.58
